In [2]:
  !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
  !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
  !apt-get update -qq 2>&1 > /dev/null
  !apt-get -y install -qq google-drive-ocamlfuse fuse
  from google.colab import auth
  auth.authenticate_user()
  from oauth2client.client import GoogleCredentials
  creds = GoogleCredentials.get_application_default()
  import getpass
  !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
  vcode = getpass.getpass()
  !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
··········


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [4]:
%cd drive
%cd Problem Cars_ Dataset
!ls

/content/drive
/content/drive/Problem Cars_ Dataset
csvfile.csv  finalized_model.sav  sample.csv  test.csv	test.xlsx  train.csv


In [0]:
import pandas as pd
import numpy as np


In [0]:
data_xls = pd.read_excel('test.xlsx', index_col=None)
data_xls.to_csv('test.csv', encoding='utf-8')
test=pd.read_csv('test.csv')
test.replace({'?': np.nan}, inplace=True)
missingtest=test.isnull().sum()

In [312]:
missingtest

Unnamed: 0           0
id                   0
fuel                 0
ABS-compatibility    0
parking-sensor       0
symboling            0
normalized-losses    5
num-of-doors         1
body-style           0
manufacturer         0
condition            0
wheel-base           0
length               0
width                0
height               0
car-weight           0
engine-type          0
num-of-persons       0
engine-size          0
battery-type         0
bore                 1
stroke               1
compression-ratio    0
horsepower           1
peak-rpm             1
city-mpg             0
highway-mpg          0
dtype: int64

In [0]:

test['bore'] = test['bore'].fillna(-100).astype(float)
test['normalized-losses'] = test['normalized-losses'].fillna(-100).astype(float)
test['stroke'] = test['stroke'].fillna(-100).astype(float)
test['horsepower'] = test['horsepower'].fillna(-100).astype(float)
test['peak-rpm'] = test['peak-rpm'].fillna(-100).astype(float)

In [0]:
cleanup_nums = {"num-of-doors":     {"four": 4, "two": 2},
                "num-of-persons": {"four": 4, "six": 6, "five": 5, "eight": 8,
                                  "two": 2, "twelve": 12, "three":3 },
                  "condition":     {"new": 1, "old": 0},
                "fuel": {"petrol":1,"diesel":0 },
               "ABS-compatibility":{"ABS not supported":0,"ABS supported":1}}

test.replace(cleanup_nums, inplace=True)


In [315]:
obj_test = test.select_dtypes(include=['object']).copy()
obj_test.head()

,parking-sensor,body-style,manufacturer,engine-type,battery-type
0,installed,hatchback,plymouth,ohc,li-ion
1,not installed,sedan,volvo,ohc,sli
2,installed,sedan,mazda,ohc,li-ion
3,installed,hatchback,toyota,ohc,li-ion
4,not installed,sedan,volvo,ohc,sli


In [0]:
onetest=one_hot_encoded_training_predictors = pd.get_dummies(obj_test)

In [0]:
bigtest=pd.concat([test, onetest],axis=1)
bigtest=bigtest.drop(['Unnamed: 0','parking-sensor','body-style','manufacturer','engine-type','battery-type'],axis=1)

In [0]:
bigtest1=bigtest.replace(-100, np.nan)

In [319]:
from fancyimpute import KNN 
train_cols = list(bigtest1)
dtest= pd.DataFrame(KNN(k=5).fit_transform(bigtest1))
dtest.columns = dtest.columns
dtest.index = dtest.index

Imputing row 1/21 with 0 missing, elapsed time: 0.003


In [0]:
dtest.columns=['id', 'fuel', 'ABS-compatibility', 'symboling', 'normalized-losses',
       'num-of-doors', 'condition', 'wheel-base', 'length', 'width', 'height',
       'car-weight', 'num-of-persons', 'engine-size', 'bore', 'stroke',
       'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg',
       'highway-mpg', 'parking-sensor_installed',
       'parking-sensor_not compatible', 'parking-sensor_not installed',
       'body-style_hardtop', 'body-style_hatchback', 'body-style_sedan',
       'body-style_wagon', 'manufacturer_audi', 'manufacturer_dodge',
       'manufacturer_mazda', 'manufacturer_nissan', 'manufacturer_peugot',
       'manufacturer_plymouth', 'manufacturer_porsche', 'manufacturer_renault',
       'manufacturer_saab', 'manufacturer_subaru', 'manufacturer_toyota',
       'manufacturer_volkswagen', 'manufacturer_volvo', 'engine-type_dohc',
       'engine-type_l', 'engine-type_ohc', 'engine-type_ohcf',
       'engine-type_rotor', 'battery-type_agm', 'battery-type_li-ion',
       'battery-type_sli', 'battery-type_vrla']

In [0]:
dtest=dtest[['wheel-base', 'length', 'width', 'car-weight', 'num-of-persons',
       'engine-size', 'bore', 'horsepower', 'city-mpg', 'highway-mpg',
       'parking-sensor_installed', 'parking-sensor_not installed',
        'battery-type_li-ion',
       'battery-type_sli']]

In [322]:
dtest

,wheel-base,length,width,car-weight,num-of-persons,engine-size,bore,horsepower,city-mpg,highway-mpg,parking-sensor_installed,parking-sensor_not installed,battery-type_li-ion,battery-type_sli
0,93.7,157.3,63.8,1918.0,4.0,90.0,2.970000,68.000000,37.0,41.0,1.0,0.0,1.0,0.0
1,104.3,188.8,67.2,3045.0,4.0,130.0,3.620000,162.000000,17.0,22.0,0.0,1.0,0.0,1.0
2,93.1,166.8,64.2,1950.0,4.0,91.0,3.080000,68.000000,31.0,38.0,1.0,0.0,1.0,0.0
3,95.7,166.3,64.4,2122.0,4.0,98.0,3.190000,70.000000,28.0,34.0,1.0,0.0,1.0,0.0
4,109.1,188.8,68.8,3049.0,4.0,141.0,3.780000,160.000000,19.0,25.0,0.0,1.0,0.0,1.0
5,96.9,173.6,65.4,2420.0,4.0,108.0,3.620000,82.000000,23.0,29.0,0.0,0.0,1.0,0.0
6,93.7,157.9,63.6,2120.0,4.0,108.0,3.620000,73.000000,26.0,31.0,1.0,0.0,1.0,0.0
7,95.3,169.0,65.7,2385.0,2.0,70.0,3.456595,101.000000,17.0,23.0,0.0,1.0,0.0,0.0
8,107.9,186.7,68.4,3197.0,4.0,152.0,3.700000,95.000000,28.0,33.0,0.0,1.0,0.0,0.0
9,97.0,172.0,65.4,2385.0,4.0,108.0,3.620000,82.000000,24.0,25.0,0.0,0.0,1.0,0.0


In [0]:
import pickle
loaded_model = pickle.load(open('finalized_model.sav', 'rb'))


In [0]:
array=loaded_model.predict(dtest)
array=array.astype(int)
array1=np.copy(array)

In [0]:
price=pd.DataFrame(array)

In [0]:

for n in range(array1.size):
   if array1[n]<=(np.percentile(array1, 25, axis=0)):
    array1[n]=(np.percentile(array, 25, axis=0))
    
   elif array1[n]<=(np.percentile(array1, 50, axis=0)):
    array1[n]=(np.percentile(array, 50, axis=0))
 
   else:
     array1[n]=(np.percentile(array, 100, axis=0))
      
 
 

In [0]:
category=pd.DataFrame(array1)


In [0]:
l=np.percentile(array, 25, axis=0)
m=np.percentile(array, 50, axis=0)
h=np.percentile(array, 100, axis=0)
category.replace({l:'Low',m:'Medium',h:'High'}, inplace=True) 


In [0]:
result = pd.concat([price, category], axis=1)

In [0]:
result.columns=['price','price-category']

In [331]:
result

,price,price-category
0,5385,Low
1,17365,High
2,5963,Low
3,6645,Low
4,19995,High
5,10776,Medium
6,5734,Low
7,8381,Medium
8,19100,High
9,9800,Medium


In [0]:
import csv

with open('output', 'wb') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)

In [0]:
result.to_csv("output")